In [6]:
import numpy as np
import random

# Value Function
3 dimensional array, where first index is \[cardsum(12 to 21) - 12\] (_for indexing_); second index is 0 if no usable ace and 1 if any; third index is dealer card \[(1 to 10)-1\](_for indexing_).

In [7]:
V = np.array([[[0.]*10]*2]*10)

In [16]:
V

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.,

# BlackJack Dynamic Programming Strategy

In [17]:
states = []
for i in range(10):
    for j in range(2):
        for k in range(10):
            states.append((i, j, k))
states

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 0, 4),
 (0, 0, 5),
 (0, 0, 6),
 (0, 0, 7),
 (0, 0, 8),
 (0, 0, 9),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 1, 6),
 (0, 1, 7),
 (0, 1, 8),
 (0, 1, 9),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 0, 5),
 (1, 0, 6),
 (1, 0, 7),
 (1, 0, 8),
 (1, 0, 9),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 1, 4),
 (1, 1, 5),
 (1, 1, 6),
 (1, 1, 7),
 (1, 1, 8),
 (1, 1, 9),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 0, 4),
 (2, 0, 5),
 (2, 0, 6),
 (2, 0, 7),
 (2, 0, 8),
 (2, 0, 9),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (2, 1, 4),
 (2, 1, 5),
 (2, 1, 6),
 (2, 1, 7),
 (2, 1, 8),
 (2, 1, 9),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 0, 4),
 (3, 0, 5),
 (3, 0, 6),
 (3, 0, 7),
 (3, 0, 8),
 (3, 0, 9),
 (3, 1, 0),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3),
 (3, 1, 4),
 (3, 1, 5),
 (3, 1, 6),
 (3, 1, 7),
 (3, 1, 8),
 (3, 1, 9),
 (4, 0, 0),
 (4, 0, 1),
 (4, 0, 2),
 (4,

In [18]:
dealer_dist = [
    [13.6, 5.6, 14.3, 14.6, 13.8, 38.1],
    [34.9, 13.6, 14.1, 12.8, 12.8, 11.9],
    [38.1, 11.8, 13.4, 12.6, 12.2, 12.0],
    [39.6, 11.9, 13.3, 12.1, 11.8, 11.4],
    [42.1, 12.1, 12.4, 11.0, 11.6, 10.9],
    [43.5, 12.3, 11.8, 11.0, 11.2, 10.3],
    [26.3, 37.0, 14.1, 7.9, 7.8, 7.0],
    [25.0, 12.7,35.6, 12.8, 7.3, 6.6],
    [22.3, 12.0, 12.0, 35.6, 11.9, 6.2],
    [21.3, 11.25, 11.5, 11.1, 34.1, 11.3]
]

# cumulative win probability
# sum of all indexes before
for probabilities in dealer_dist:
    for i in range(1, 6):
        probabilities[i] += probabilities[i-1]
        
dealer_dist

[[13.6, 19.2, 33.5, 48.1, 61.900000000000006, 100.0],
 [34.9, 48.5, 62.6, 75.4, 88.2, 100.10000000000001],
 [38.1,
  49.900000000000006,
  63.300000000000004,
  75.9,
  88.10000000000001,
  100.10000000000001],
 [39.6, 51.5, 64.8, 76.89999999999999, 88.69999999999999, 100.1],
 [42.1, 54.2, 66.60000000000001, 77.60000000000001, 89.2, 100.10000000000001],
 [43.5, 55.8, 67.6, 78.6, 89.8, 100.1],
 [26.3, 63.3, 77.39999999999999, 85.3, 93.1, 100.1],
 [25.0, 37.7, 73.30000000000001, 86.10000000000001, 93.4, 100.0],
 [22.3, 34.3, 46.3, 81.9, 93.80000000000001, 100.00000000000001],
 [21.3, 32.55, 44.05, 55.15, 89.25, 100.55]]

In [19]:
def blackjack_probability(s_, s, a, r):
    stay = 0
    hit = 1
      
    
    if a == stay:
        '''find Probability of win, loss, tie'''
        if s[0] < 5:
            # probability dealer busts
            if r == 1:
                return (dealer_dist[s[2]][0])/100

            else:
                return (1 - dealer_dist[s[2]][0]/100)
        
        else:
            index = s[0] - 4      # 17 is 1
            if r == 1:
                return (dealer_dist[s[2]][index-1]/ 100)
            
            elif r == 0:
                return ((dealer_dist[s[2]][index] - dealer_dist[s[2]-1][index-1])/100)
            
            else:
                return (1-(dealer_dist[s[2]][index]/100))
                
    
    if a == hit:
        '''Find probability s_'''
        if s_[2] != s_[2]:
            '''dealer card is not the same'''
            return 0
        
        if s_ == (-1, -1, -1):
            '''find Probability of a bust'''
            return (4 + s[0])/13
            
        if s_[1] == 0 and s[1] == 1:
            '''player has to use his ace'''
            if s_[0] == s[0]:
                '''probability of a 10 value card'''
                return 4/13
            
            if s_[0] > s[0]:
                return 0
            
            return 1/13
        
        if s_[1] == s[1]:
            '''no ace transaction'''
            if s_[0] <= s[0]:
                '''new hand cannot be less than old hand'''
                return 0
        
        return 1/13

In [20]:
def pi_20(action, state):
    '''deterministic policy that returns the probability of action given state'''
    stay = 0
    hit = 1
    
    if state[0] >= 8:
        if action == stay:  
            return 1
        else:
            return 0
    else:
        if action == hit:
            return 0
        else:
            return 1

In [21]:
def evaluate_policy(V, theta):
    while True:
        delta = 0
        for s1, s2, s3 in states:
            v = V[s1][s2][s3]
            bellman_update(V, (s1, s2, s3))
            delta = max(delta, abs(v - V[s1][s2][s3]))
        if delta < theta:
            break
    return V

In [22]:
def bellman_update(V, state):
    """Mutate ``V`` according to the Bellman update equation."""
    stay = 0
    hit = 1
    
    hit_value = 0
    for s_ in states:
        Vs_ = V[s_[0]][s_[1]][s_[2]]
        hit_value += blackjack_probability(s_, state, hit, 0)*(0 + Vs_) 
    
    # probability of a bust
    hit_value += blackjack_probability((-1, -1, -1), state, hit, 0)*(-1 + 0) 
    
    # value of staying
    stay_value = 0
    for r in range(3):
        stay_value += blackjack_probability((-1, -1, -1), state, stay, (r-1))*(r-1)
    
    hand, ace, dealer = state
    V[hand][ace][dealer] = pi_20(1, state)*hit_value + pi_20(0, state)*stay_value
    return 

In [15]:
evaluate_poli

NameError: name 'evaluate_poli' is not defined

In [25]:
V

array([[[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0

In [24]:
evaluation = evaluate_policy(V, 0.002)

In [32]:
import plotly.express as px

fig = px.imshow(V)
fig.show()

ValueError: px.imshow only accepts 2D single-channel, RGB or RGBA images. An image of shape (10, 2, 10) was provided

In [35]:
without_ace

[[0.6190000000000001,
  0.8830000000000001,
  0.8820000000000002,
  0.8879999999999998,
  0.8930000000000001,
  0.8989999999999999,
  0.9319999999999998,
  0.934,
  0.9380000000000004,
  0.898],
 [0.6190000000000001,
  0.8830000000000001,
  0.8820000000000002,
  0.8879999999999998,
  0.8930000000000001,
  0.8989999999999999,
  0.9319999999999998,
  0.934,
  0.9380000000000004,
  0.898],
 [0.6190000000000001,
  0.8830000000000001,
  0.8820000000000002,
  0.8879999999999998,
  0.8930000000000001,
  0.8989999999999999,
  0.9319999999999998,
  0.934,
  0.9380000000000004,
  0.898],
 [0.6190000000000001,
  0.8830000000000001,
  0.8820000000000002,
  0.8879999999999998,
  0.8930000000000001,
  0.8989999999999999,
  0.9319999999999998,
  0.934,
  0.9380000000000004,
  0.898],
 [0.6190000000000001,
  0.8830000000000001,
  0.8820000000000002,
  0.8879999999999998,
  0.8930000000000001,
  0.8989999999999999,
  0.9319999999999998,
  0.934,
  0.9380000000000004,
  0.898],
 [0.6190000000000001,
  0

In [33]:
V

array([[[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ]],

       [[-0.728 , -0.302 , -0.238 , -0.208 , -0.158 , -0.13  , -0.474 ,
         -0.5   , -0.554 , -0.574 ],
        [-0

In [28]:
with_ace = [[0.]*10]*10
for i in range(len(V)):
    for j in range(len(V[0][0])):
         with_ace[i][j] = V[i][1][j]

In [26]:
without_ace = [[0.]*10]*10
for i in range(len(V)):
    for j in range(len(V[0][0])):
         without_ace[i][j] = V[i][0][j]